In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import warnings 
warnings.filterwarnings('ignore')
from arguments import parser
from datasets import create_dataset
from easydict import EasyDict
from softpatch.main import get_coreset,get_sampler

from torch.utils.data import DataLoader 

default_setting = './configs/benchmark/pc_mvtecad.yaml'
# default_setting = './configs/benchmark/pc_mvtecloco.yaml'
cfg = parser(jupyter=True, default_setting = default_setting)
cfg.DATASET.params.baseline = True 

# load dataset
trainset, testset = create_dataset(
    dataset_name  = cfg.DATASET.dataset_name,
    datadir       = cfg.DATASET.datadir,
    class_name    = cfg.DATASET.class_name,
    img_size      = cfg.DATASET.img_size,
    mean          = cfg.DATASET.mean,
    std           = cfg.DATASET.std,
    aug_info      = cfg.DATASET.aug_info,
    **cfg.DATASET.get('params',{})
)

trainloader = DataLoader(trainset,batch_size = 32,shuffle=False)
testloader = DataLoader(testset, batch_size = 32, shuffle=True)


In [ ]:
import timm 
import torch 
import torch.nn as nn 
from softpatch.src import common 
from models import PatchCore

model = PatchCore(
    backbone               = 'wide_resnet50_2',
    layers_to_extract_from = cfg.MODEL.params.layers_to_extract_from,
    input_shape            = (3,cfg.DATASET.img_size, cfg.DATASET.img_size),
    sampler_name           = cfg.MODEL.params.sampler_name,
    sampling_ratio         = cfg.MODEL.params.sampling_ratio,
    faiss_on_gpu           = cfg.MODEL.params.faiss_on_gpu,
    faiss_num_workers      = cfg.MODEL.params.faiss_num_workers,
    lof_k                  = cfg.MODEL.params.lof_k,
    threshold              = cfg.MODEL.params.threshold,
    weight_method          = cfg.MODEL.params.weight_method,
    with_soft_weight       = cfg.MODEL.params.with_soft_weight,
    device                 = f"cuda:{cfg.MODEL.params.device}"
    )                        

In [3]:
method = cfg.MODEL.method
backbone = cfg.MODEL.backbone 
model_params = cfg.MODEL.params

model = __import__('models').__dict__[method](
                           backbone = backbone,
                           **model_params
                           )

In [4]:
for imgs, labels, gts in trainloader:
    model(imgs)
model.fit()

Subsampling...: 100%|██████████| 42924/42924 [00:23<00:00, 1816.00it/s]


In [5]:
import numpy as np 
import torch 
from torchmetrics import AUROC 

img_auroc = AUROC()
pix_auroc = AUROC()
for imgs, labels, gts in testloader:
    scores, masks = model.get_score_map(imgs.to('cuda'))
    
    img_auroc.update(torch.Tensor(scores), labels)
    pix_auroc.update(torch.Tensor(masks).reshape(-1),gts.reshape(-1).type(torch.int8))
    
print(img_auroc.compute(), pix_auroc.compute())

tensor(0.9988) tensor(0.9895)


In [1]:
from torchmetrics import AUROC 
from sklearn.metrics import precision_recall_curve, roc_curve, auc, confusion_matrix
from utils.metrics import compute_pro
def calculate_aupro(anomaly_maps:np.ndarray, ground_truth_maps:np.ndarray):    
    # 모든 이미지에 대한 FPR과 PRO 값을 저장할 리스트
    all_fprs = []
    all_pros = []

    # 각 이미지에 대해 compute_pro 호출
    aupro = [] 
    for i in range(len(anomaly_maps)):
        fprs, pros = compute_pro(anomaly_maps[i], ground_truth_maps[i])
        aupro.append(auc(fprs,pros))

    # 전체 FPR과 PRO 값들을 정렬
    # sorted_indices = np.argsort(all_fprs)
    # sorted_fprs = np.array(all_fprs)[sorted_indices]
    # sorted_pros = np.array(all_pros)[sorted_indices]

    return np.mean(aupro)

class MetricCalculator:
    '''
    metric = MetricCalculator(metric_list=['auroc])
    
    metric.update(preds:torch.Tensor, target:torch.Tensor)
    '''
    def __init__(self, metric_list: list):
        self.metric_list = metric_list 
        self.preds = [] 
        self.targets = [] 
        
    def _confusion_matrix(self, y_preds:np.ndarray, y_trues:np.ndarray):
        y_preds, y_trues = y_preds.flatten(), y_trues.flatten()
        fpr, tpr, thr = roc_curve(y_trues,y_preds)
        cm_list = {} 
        for fpr_ratio in [0.005, 0.01, 0.05, 0.1]:
            fpr_thr = thr[np.argmin(fpr < fpr_ratio)-1]
            y_preds = np.where(y_preds > fpr_thr,1,0)
            cm = confusion_matrix(y_trues, y_preds)
            
            cm_list[fpr_ratio] = cm 
        return cm_list 
    
    def _detach(self, data):
        if isinstance(data, torch.Tensor):
            out = data.detach().cpu().numpy()
        elif isinstance(data, np.ndarray):
            out = data 
        return out 
    
    def _average_precision(self, y_preds:np.ndarray, y_trues:np.ndarray):
        y_preds, y_trues = y_preds.flatten(), y_trues.flatten()
        precision, recall, _ = precision_recall_curve(y_trues, y_preds)
        aupr = auc(recall, precision)
        return aupr 
    
    def _auroc(self, y_preds:np.ndarray, y_trues:np.ndarray):
        y_preds, y_trues = y_preds.flatten(), y_trues.flatten()
        fpr, tpr, thr = roc_curve(y_trues,y_preds)
        auroc = auc(fpr,tpr)
        return auroc
    
    def _aupro(self, y_preds:np.ndarray, y_trues:np.ndarray):
        fprs, pros = calculate_aupro(y_preds, y_trues)
        aupro = auc(fprs,pros)
        return aupro 
    
    def update(self, y_preds:torch.Tensor, y_trues:torch.Tensor) -> None:
        '''
        preds : torch.Tensor -> np.ndarray -> append in list 
        '''
        self.preds.append(
            self._detach(y_preds)
        )
        self.targets.append(
            self._detach(y_trues)
        )
        
        
    def compute(self):        
        y_preds = np.concatenate(self.preds) # (N,W,H)
        y_trues = np.concatenate(self.targets) # (N,W,H)
        
        result_list = {}         
        for metric in self.metric_list:
            result_list[metric] = eval(f'self._{metric}')(y_preds, y_trues)            
        return result_list 

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'np' is not defined

In [5]:
import torch 
torch.Tensor([10,20,30])

tensor([10., 20., 30.])

In [2]:
metric = MetricCalculator(
    metric_list = ['auroc','average_precision']
)

/usr/local/lib/python3.8/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [ ]:
metric.update(torch.Tensor(scores), labels)


In [ ]:
result = metric.compute()

In [ ]:
result

In [ ]:
result